In [3]:
!pip install pandas
!pip install tqdm
!pip install -U scikit-learn scipy matplotlib


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.8/dist-packages (1.0.2)
Requirement already up-to-date: scipy in /usr/local/lib/python3.8/dist-packages (1.8.0)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.8/dist-packages (3.5.1)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
import os

import datetime as dt
import pandas as pd
import numpy as np
import pandas as pd
import tqdm
import random
import pylab

from datetime import datetime, timedelta
from dateutil import parser
from numpy import array
from tensorflow.keras.layers import Dense, LSTM,GRU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Sequential
from collections import Counter
from pandas import DataFrame
from enum import Enum




In [5]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]

        seq_x = np.pad(seq_x, ((0, 0), (0, 30 - seq_x.shape[1])), 'constant')
        X.append(seq_x)
        y.append(seq_y[-1])
    return array(X), array(y)

In [6]:
from matplotlib import pyplot as plt
def draw_timeline(name,vulns,first_date, last_date):

    dates = vulns
    dates += [first_date]
    dates += [last_date]

    values = [1]*len(dates)
    values[-1] = 2
    values[-2] = 2

    X = pd.to_datetime(dates)
    fig, ax = plt.subplots(figsize=(6,1))
    ax.scatter(X, [1]*len(X), c=values,
               marker='s', s=100)
    fig.autofmt_xdate()

    # everything after this is turning off stuff that's plotted by default
    ax.set_title(name)
    ax.yaxis.set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['top'].set_visible(True)
    ax.xaxis.set_ticks_position('bottom')
    ax.set_facecolor('white')
    
    ax.get_yaxis().set_ticklabels([])
    # day = pd.to_timedelta("1", unit='D')
    # plt.xlim(X[0] - day, X[-1] + day)
    plt.show()
    #plt.subplots_adjust(bottom=0.15)
    #plt.savefig(f"D:/cve/images/timeline/{name}.jpg", transparent=False)
    
    
def find_benign_events(cur_repo_data,gap_days, num_of_events):
    benign_events = []
    retries = num_of_events * 5
    counter = 0
    for _ in range(num_of_events):
        found_event = False
        while not found_event:
            if counter >=retries:
                return benign_events
            try:
                cur_event = random.randint(2*gap_days+1,cur_repo_data.shape[0]-gap_days*2-1)
            except ValueError:
                counter +=1
                continue
            event = cur_repo_data.index[cur_event]

            before_vuln = event - gap_days
            after_vuln = event + gap_days
            res_event = cur_repo_data[before_vuln:event-1]
            if not res_event[res_event["VulnEvent"]>0].empty:
                counter +=1
                continue
            benign_events.append(res_event.iloc[:,:-1].values)
            found_event = True
            
            
    return benign_events

def create_all_events(cur_repo_data,gap_days):
    all_events = []
    labels = []
    for i in range(gap_days,cur_repo_data.shape[0],1):
            event = cur_repo_data.index[i]
            before_vuln = event - gap_days
            res_event = cur_repo_data[before_vuln:event-1]
            all_events.append(res_event.iloc[:,:-1].values)
            labels.append(res_event.iloc[:,-1].values)
    return all_events,labels


In [9]:
repo_dirs = '../repo_gharchive_processed4'
benign_all, vuln_all = [], []
n_features = 0
gap_days = 150

nice_list= ['facebook_hhvm.csv',
'ffmpeg_ffmpeg.csv',
'flatpak_flatpak.csv',
'freerdp_freerdp.csv',
'git_git.csv',
'gpac_gpac.csv',
'imagemagick_imagemagick.csv',
'kde_kdeconnect-kde.csv',
'krb5_krb5.csv',
'mantisbt_mantisbt.csv',
'op-tee_optee_os.csv',
'owncloud_core.csv',
'php_php-src.csv',
'revive-adserver_revive-adserver.csv',
'rubygems_rubygems.csv',
'the-tcpdump-group_tcpdump.csv']

class Aggregate(Enum):
    none = 1
    before_cve = 2
    after_cve = 3
    
aggr_option = Aggregate.after_cve

for file in tqdm.tqdm(os.listdir(repo_dirs)[:]):
    try:
        try:
            selected = 4
            # if file != nice_list[selected]:
            #          continue
            cur_repo_data = pd.read_csv(repo_dirs + "/" + file,parse_dates=['created_at'],index_col='created_at')
            
            #    pass #all_events,labels = create_all_events(cur_repo_data,gap_days)
                #correct_labels = [label[-1] for label in labels]
            # if cur_repo_data[cur_repo_data["VulnEvent"] > 0].values.shape[0]<5: 
            #     continue
                
            cur_repo_data = cur_repo_data[cur_repo_data.index.notnull()]
            cur_repo_data["additions"]=(cur_repo_data["additions"]-cur_repo_data["additions"].mean())/cur_repo_data["additions"].std()
            cur_repo_data["deletions"]=(cur_repo_data["deletions"]-cur_repo_data["deletions"].mean())/cur_repo_data["deletions"].std()
            
            y = pd.get_dummies(cur_repo_data.index.day_of_week, prefix='day_of_week')
            y.index = cur_repo_data.index
            cur_repo_data = pd.concat([cur_repo_data,y],axis=1)
            y = pd.get_dummies(cur_repo_data.index.hour, prefix='hour')
            y.index = cur_repo_data.index
            cur_repo_data = pd.concat([cur_repo_data,y],axis=1)
            
            if aggr_option == Aggregate.before_cve:
                cur_repo_data = cur_repo_data.resample('W-MON').sum()
            if aggr_option != Aggregate.after_cve:
                cur_repo_data = cur_repo_data.reset_index(drop=True)


        except pd.errors.EmptyDataError:
            #print(file)
            continue
        if cur_repo_data.shape[0]<100:
            continue
        cols_at_end = ['VulnEvent']
        cur_repo_data = cur_repo_data[[c for c in cur_repo_data if c not in cols_at_end]
                                + [c for c in cols_at_end if c in cur_repo_data]]
        # vulns = cur_repo_data[cur_repo_data["VulnEvent"] > 0]
        
        
        # if file == nice_list[selected]:
        #     all_events,labels = create_all_events(cur_repo_data,gap_days)
        
        
        vulns = cur_repo_data.index[cur_repo_data['VulnEvent'] > 0].tolist()

        # vulns = vulns.sort_index()
        
        for vuln in vulns:
            # before,after = vuln - timedelta(days=gap_days),vuln + timedelta(days=gap_days)
            before, after = vuln-gap_days,vuln+gap_days
            res = cur_repo_data[before:vuln-1].iloc[:,:-1].values
            if aggr_option == Aggregate.after_cve:
                cur_repo_data = cur_repo_data.resample('W-MON').sum()
            vuln_all.append(res)
        break
        num_of_events = 7*len(vulns)
        benigns=find_benign_events(cur_repo_data,gap_days,num_of_events)

        for res in benigns:
            benign_all.append(res)

        """for vuln in vulns.index:
            before_vuln = vuln - timedelta(days=gap_days)
            after_vuln = vuln + timedelta(days=gap_days)
            # draw_timeline(file.split(".csv")[0],list(vulns.index),cur_data.index[0],cur_data.index[-1])
            cur_data = cur_repo_data[before_vuln:after_vuln]


            #cur_train_x, cur_train_y = split_sequence(cur_data.values, n_steps)

            # if len(cur_train_x) == 0:
            #     continue

            all_train_x.append(cur_data.values)
            all_train_y.append([1])

        for benign in find_benign_events(cur_repo_data,gap_days,num_of_events):
            print(benign.shape)
            all_train_x.append(benign)
            all_train_y.append([0])"""
        #print(len(all_train_x),len(all_train_y))
    except KeyError as e:
        #print(file)
        continue
        # import traceback
        # traceback.print_exc()
        

# all_train_x = np.array(all_train_x)
# all_train_y = np.array(all_train_y)
# print(all_train_x.shape,all_train_y.shape)
# np.save('x.npy', all_train_x, allow_pickle=True)
# np.save('y.npy', all_train_y, allow_pickle=True)


  0%|          | 0/1784 [00:00<?, ?it/s]


TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [154]:
print("HI")

max_vals = max(Counter([v.shape for v in vuln_all]))
vuln_all = [v for v in vuln_all if v.shape == max_vals]
max_vals = max(Counter([v.shape for v in benign_all]))
benign_all = [v for v in benign_all if v.shape == max_vals]

vuln_all =np.nan_to_num(np.array(vuln_all))
benign_all = np.nan_to_num(np.array(benign_all)) 


np.save('vuln_week_150back_7_1.npy', np.array(vuln_all))    # .npy extension is added if not given
np.save('benign_week_150back_7_1.npy', np.array(benign_all))    # .npy extension is added if not given

HI


In [155]:

vuln_all = np.load("vuln_week_150back_7_1.npy")
benign_all = np.load("benign_week_150back_7_1.npy")
def normalize(time_series_feature):
    if time_series_feature.max()-time_series_feature.min() == 0:
        return time_series_feature
    return (time_series_feature-time_series_feature.min())/(time_series_feature.max()-time_series_feature.min())

In [156]:


all_train_x = np.concatenate([vuln_all,benign_all])
all_train_y = np.concatenate([np.ones(vuln_all.shape[0]),np.zeros(benign_all.shape[0])])
all_train_x.shape,all_train_y.shape

NORMALIZE = False

if NORMALIZE:
    all_train_x= normalize(all_train_x)
    vuln_all2 = normalize(vuln_all)
    benign_all2 = normalize(benign_all)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(all_train_x.shape, all_train_x[0].shape)
print(all_train_y.shape, all_train_y[0].shape)
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras import Input, layers
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = train_test_split(all_train_x,all_train_y,shuffle=True)


RESAMPLE = False
if RESAMPLE:
    ros = RandomOverSampler(random_state=42)
    ros.fit_resample(X_train[:,:,0], y_train)
    X_train = X_train[ros.sample_indices_]
    y_train = y_train[ros.sample_indices_]

model = Sequential()
#model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

reshaped_train,reshaped_test = X_train.reshape(X_train.shape[0],-1), X_test.reshape(X_test.shape[0],-1)

# define model
model = Sequential()
model.add(Conv1D(filters=100, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.8))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(70, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', mode='max',patience=30)

history = model.fit(X_train, y_train, verbose=1,epochs=300, batch_size=32,validation_data=(X_test,y_test),callbacks=[])

# print(model.evaluate(X_test.reshape(X_test.shape[0],-1), y_test, verbose=0))
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'val'], loc='upper left')
pyplot.show()

# model.fit(benign_all2, benign_all2, epochs=100)
          
"""
threshold = find_threshold(model, benign_all2)
print(f"Threshold: {threshold}")
# Threshold: 0.01001314025746261
predictions = get_predictions(model, vuln_all2, threshold)
accuracy_score(predictions, [0]*len(vuln_all2))"""

(12041, 149, 54) (149, 54)
(12041,) ()
Epoch 1/300
283/283 [==============================] - 8s 25ms/step - loss: 1.1110 - accuracy: 0.6913 - val_loss: 0.5278 - val_accuracy: 0.7818
Epoch 2/300
283/283 [==============================] - 8s 30ms/step - loss: 0.5960 - accuracy: 0.7779 - val_loss: 0.5089 - val_accuracy: 0.7705
Epoch 3/300
283/283 [==============================] - 10s 34ms/step - loss: 0.5343 - accuracy: 0.7956 - val_loss: 0.4536 - val_accuracy: 0.8107
Epoch 4/300
283/283 [==============================] - 10s 37ms/step - loss: 0.4777 - accuracy: 0.8192 - val_loss: 0.4366 - val_accuracy: 0.8256
Epoch 5/300
283/283 [==============================] - 10s 37ms/step - loss: 0.4617 - accuracy: 0.8454 - val_loss: 0.3809 - val_accuracy: 0.8525
Epoch 6/300
283/283 [==============================] - 10s 37ms/step - loss: 0.4095 - accuracy: 0.8559 - val_loss: 0.3778 - val_accuracy: 0.8582
Epoch 7/300
283/283 [==============================] - 10s 37ms/step - loss: 0.3962 - accurac

Epoch 57/300
283/283 [==============================] - 11s 38ms/step - loss: 0.2790 - accuracy: 0.9075 - val_loss: 0.3204 - val_accuracy: 0.8871
Epoch 58/300
283/283 [==============================] - 11s 38ms/step - loss: 0.2852 - accuracy: 0.9061 - val_loss: 0.3475 - val_accuracy: 0.8728
Epoch 59/300
283/283 [==============================] - 11s 38ms/step - loss: 0.2922 - accuracy: 0.9016 - val_loss: 0.3028 - val_accuracy: 0.8921
Epoch 60/300
283/283 [==============================] - 11s 38ms/step - loss: 0.2856 - accuracy: 0.9050 - val_loss: 0.3397 - val_accuracy: 0.8804
Epoch 61/300
283/283 [==============================] - 11s 39ms/step - loss: 0.2846 - accuracy: 0.9051 - val_loss: 0.3246 - val_accuracy: 0.8894
Epoch 62/300
283/283 [==============================] - 11s 39ms/step - loss: 0.2760 - accuracy: 0.9096 - val_loss: 0.3120 - val_accuracy: 0.8934
Epoch 63/300
283/283 [==============================] - 11s 39ms/step - loss: 0.2702 - accuracy: 0.9114 - val_loss: 0.3094 -

283/283 [==============================] - 11s 39ms/step - loss: 0.2641 - accuracy: 0.9153 - val_loss: 0.3075 - val_accuracy: 0.8970
Epoch 114/300
283/283 [==============================] - 11s 40ms/step - loss: 0.2683 - accuracy: 0.9188 - val_loss: 0.3059 - val_accuracy: 0.8987
Epoch 115/300
283/283 [==============================] - 11s 40ms/step - loss: 0.2598 - accuracy: 0.9188 - val_loss: 0.3184 - val_accuracy: 0.8951
Epoch 116/300
283/283 [==============================] - 12s 41ms/step - loss: 0.2618 - accuracy: 0.9168 - val_loss: 0.3061 - val_accuracy: 0.8984
Epoch 117/300
283/283 [==============================] - 11s 40ms/step - loss: 0.2509 - accuracy: 0.9204 - val_loss: 0.3012 - val_accuracy: 0.8990
Epoch 118/300
283/283 [==============================] - 11s 40ms/step - loss: 0.2555 - accuracy: 0.9182 - val_loss: 0.3063 - val_accuracy: 0.8997
Epoch 119/300
283/283 [==============================] - 11s 39ms/step - loss: 0.2594 - accuracy: 0.9189 - val_loss: 0.3039 - val_ac

Epoch 169/300
283/283 [==============================] - 20s 72ms/step - loss: 0.2406 - accuracy: 0.9279 - val_loss: 0.3254 - val_accuracy: 0.8874
Epoch 170/300
283/283 [==============================] - 24s 86ms/step - loss: 0.2446 - accuracy: 0.9245 - val_loss: 0.3073 - val_accuracy: 0.8987
Epoch 171/300
283/283 [==============================] - 18s 65ms/step - loss: 0.2431 - accuracy: 0.9251 - val_loss: 0.3164 - val_accuracy: 0.8897
Epoch 172/300
283/283 [==============================] - 15s 54ms/step - loss: 0.2538 - accuracy: 0.9227 - val_loss: 0.3258 - val_accuracy: 0.8937
Epoch 173/300
283/283 [==============================] - 15s 55ms/step - loss: 0.2735 - accuracy: 0.9237 - val_loss: 0.3147 - val_accuracy: 0.8977
Epoch 174/300
283/283 [==============================] - 15s 54ms/step - loss: 0.2467 - accuracy: 0.9256 - val_loss: 0.2931 - val_accuracy: 0.9037
Epoch 175/300
283/283 [==============================] - 18s 64ms/step - loss: 0.2381 - accuracy: 0.9296 - val_loss: 0

In [ ]:
all_events = np.array(all_events)
all_events.shape
df = DataFrame(model.predict(all_events),columns=['pred'])

In [ ]:
# model.save("models/150back2000epochs_84valaccuracy")

In [ ]:
df['pred2'] = df['pred'].apply(lambda x: 0.0 if x < 0.3 else 1.0)
df['real']=[int(label[-1]>0) for label in labels]
df[['pred','real']].iloc[:].plot()

pyplot.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

def find_best_f1(X_test,y_test,model):
    max_f1 = 0
    thresh = 0
    best_y = 0
    pred = model.predict(X_test)
    for i in range(100):
        y_predict = (pred.reshape(-1)>i/100).astype(int)
        precision, recall, fscore, support = score(y_test,y_predict ,zero_division=0)
        cur_f1 = fscore[1]
        print(i,cur_f1)
        if cur_f1 > max_f1:
            max_f1 = cur_f1
            best_y = y_predict
            thresh = i / 100
    return max_f1,thresh, best_y


f1,thresh,best_y = find_best_f1(X_test,y_test,model)
print(f1)



In [ ]:
print(model.evaluate(X_test.reshape(X_test.shape[0],-1), y_test, verbose=0))


In [ ]:
unshaped = X_test.reshape(X_test.shape[0],-1)

a = model.predict(unshaped)
from pandas import DataFrame
DataFrame(a).plot()

In [ ]:
res = pd.DataFrame(unshaped)
res['predict']=a
res['real']=y_test
res2=res

In [ ]:
res2=res
res2['predict']=res2['predict'].apply(lambda x: 0.0 if x<0.5 else 1.0)
res2[res['predict']!=res['real']]

In [ ]:
os.mkdir(models)
model.save(f'models/serial_1_4_f1_{f1}.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
import tensorflow as tf
nb_epoch = 10
batch_size = 64
input_dim = benign_all2.shape[1] #num of columns, 30
encoding_dim = 14
hidden_dim_1 = int(encoding_dim / 2) #
hidden_dim_2=4  
learning_rate = 1e-7

#input Layer
input_layer = tf.keras.layers.Input(shape=(input_dim, ))
#Encoder
encoder = tf.keras.layers.Dense(encoding_dim, activation="tanh",                                activity_regularizer=tf.keras.regularizers.l2(learning_rate))(input_layer)
encoder=tf.keras.layers.Dropout(0.2)(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_2, activation=tf.nn.leaky_relu)(encoder)
# Decoder
decoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
decoder=tf.keras.layers.Dropout(0.2)(decoder)
decoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(input_dim, activation='tanh')(decoder)
#Autoencoder
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
#autoencoder.summary()

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

history = autoencoder.fit(benign_all2, benign_all2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(vuln_all2, vuln_all2),
                    verbose=1,
                    ).history

In [ ]:
vuln_all2.shape

In [ ]:
mall = np.concatenate([benign_all2,vuln_all2]) # .shape ,benign_all2.shape,vuln_all2.shape

In [ ]:

threshold_fixed = 0.1
check = vuln_all2
#mall = check
test_x_predictions1 = autoencoder.predict(vuln_all2)
# test_x_predictions2 = autoencoder.predict(mall[int(mall.shape[0]/2):])


In [ ]:
mse = np.mean(np.power(mall - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': len(benign_all2)*[0]+len(vuln_all2)*[1]})

In [ ]:
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(0.4, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
# plt.ylim([0,1])
plt.title("Reconstruction error for normal and fraud data")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show()

In [ ]:
accuracy_score(predictions, [1]*len(vuln_all2))

In [ ]:
test_x_predictions = autoencoder.predict(test_data)
mse = np.mean(np.power(test_data - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_labels})
Plotting the test data points and their respective reconstruction error sets a threshold value to visualize if the threshold value needs to be adjusted.
threshold_fixed = 50
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold_fixed, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for normal and fraud data")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();


from sklearn.metrics import accuracy_score
import tensorflow as tf
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
    # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
    # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_test_scaled, threshold):
    predictions = model.predict(x_test_scaled)
    # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_test_scaled)
    # 0 = anomaly, 1 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 0.0 if x == True else 1.0)
    return preds



In [ ]:
predictions = get_predictions(model, vuln_all2, threshold-0.000011)
accuracy_score(predictions, [0]*len(vuln_all2))

In [ ]:
for i in range(30):
    print(i,np.isnan(all_train_x[:,:,i]).any())
    

In [ ]:
all_train_x = all_train_x[:,:,:21]

In [ ]:
"""
model = Sequential()

model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,validation_data=(X_test,y_test),verbose=1)
"""

"""model = Sequential()

model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5,validation_data=(X_test,y_test),verbose=2)


"""



"""model = Sequential()
model.add(Dense(50, input_shape=[all_train_x.shape[1]], activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.1), metrics=['accuracy'])

"""

"""model = Sequential()
model.add(Dense(150,input_shape=[benign_all2.shape[1]], activation='linear')) 
model.add(Dense(75, activation="linear"))
model.add(Dense(10, activation="linear"))
model.add(Dense(75, activation="linear"))
model.add(Dense(benign_all2.shape[1], activation="linear"))
model.compile(optimizer='adam',loss='mse')
"""


